In [ ]:
from petastorm import make_reader
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import kurtosis,pearsonr
from scipy.signal import periodogram
from ecgdetectors import Detectors
import pywt
from sklearn.metrics import confusion_matrix,auc
import sys
from sklearn import metrics
import pandas as pd
import seaborn as sn
import os
import warnings
sys.path.append(os.path.join(os.getcwd(), ".."))
from Metrics import TSD_cal as TSD
from Metrics import HurstExponent as Hurst
from shared_utils import Statistical_test_index as STTI
from Metrics import MoRE_index_2011 as MoRE_2011
from Metrics import ECG_Assess_2022
from Metrics import Fiducial_metrics,Non_Fiducial_metrics,Our_SQA_method
warnings.simplefilter(action="ignore", category=RuntimeWarning)
path_formatted_glasgow = "/workspaces/maitrise/data/20221006_physio_quality/set-a/dataParquet"
path_petastorm = f"file:///{path_formatted_glasgow}"
path_csv_ref_label = "/workspaces/maitrise/data/20221006_physio_quality/set-a/REFERENCE.csv"

In [ ]:
from mpl_toolkits import mplot3d
%matplotlib widget
def The_3D_plot_creator(Y1,Y2,Y3,correct_label,name_1,name_2,name_3,name_label_used = "Original",semilog = False):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    for lab,col in zip(["acceptable","unacceptable"],["r","b"]):
        Y1_sel = Y1[correct_label == lab].copy()
        Y2_sel = Y2[correct_label == lab].copy()
        Y3_sel = Y3[correct_label == lab].copy()
        ax.scatter3D(Y1_sel,Y2_sel,Y3_sel,color = col,label = lab,alpha = 0.3)
    ax.legend(["acceptable","unacceptable"])
    ax.grid()
    ax.set_xlabel(name_1)
    ax.set_ylabel(name_2)
    ax.set_zlabel(name_3)

    if semilog : 
        plt.semilogy()
    plt.title(f"{name_1} VS {name_2} VS {name_3} with the {name_label_used} labelisation" )
    plt.show()


In [ ]:
##Use of Cross validation for determining optimal Threshold wPMF: 

wPMF_STTI = STTI.Statistic_reader(path_petastorm,Non_Fiducial_metrics.wPMF_score,"wPMF",[-1,2],opposite = True)
wPMF_STTI.CrossValidation_index_opt_thresh()
wPMF_STTI.Optimal_threshold_calculator()
wPMF_STTI.PR_index()
wPMF_STTI.ROC_index()
wPMF_STTI.Accuracy_calculator()
wPMF_STTI.ROC_PR_curve_homemade()
param_wPMF = wPMF_STTI.ROC_PR_curve_sklearn()
df_wPMF =  wPMF_STTI.Classification_Report_index()

In [ ]:
SNR_N_STTI = STTI.Statistic_reader(path_petastorm,Non_Fiducial_metrics.SNR_index,"Norm SNRECG",[-1,2],normalization= True)
SNR_N_STTI.CrossValidation_index_opt_thresh()
SNR_N_STTI.Optimal_threshold_calculator()
SNR_N_STTI.ROC_index()
SNR_N_STTI.PR_index()
SNR_N_STTI.Accuracy_calculator()
SNR_N_STTI.ROC_PR_curve_homemade()
param_SNR_N = SNR_N_STTI.ROC_PR_curve_sklearn()
df_SNR_N =  SNR_N_STTI.Classification_Report_index()

In [ ]:
f_STTI = STTI.Statistic_reader(path_petastorm,Fiducial_metrics.flatline_score,"Flatline",[-1,2],normalization = True,opposite = True)
f_STTI.CrossValidation_index_opt_thresh()
f_STTI.Optimal_threshold_calculator()
f_STTI.ROC_index()
f_STTI.PR_index()
f_STTI.Accuracy_calculator()
f_STTI.ROC_PR_curve_homemade()
param_f = f_STTI.ROC_PR_curve_sklearn()
df_f =  f_STTI.Classification_Report_index()

In [ ]:
TSD_STTI = STTI.Statistic_reader(path_petastorm,TSD.TSD_index,"TSD",[-1,2],normalization = True,opposite = True)
TSD_STTI.CrossValidation_index_opt_thresh()
TSD_STTI.Optimal_threshold_calculator()
TSD_STTI.ROC_index()
TSD_STTI.PR_index()
TSD_STTI.Accuracy_calculator()
TSD_STTI.ROC_PR_curve_homemade()
param_TSD = TSD_STTI.ROC_PR_curve_sklearn()
df_TSD =  TSD_STTI.Classification_Report_index()

In [ ]:
M_STTI = STTI.Statistic_reader(path_petastorm,Fiducial_metrics.Morph_score,"Intralead",[-1,3],opposite = True)
M_STTI.CrossValidation_index_opt_thresh()
M_STTI.Optimal_threshold_calculator()
M_STTI.ROC_index()
M_STTI.PR_index()
M_STTI.Accuracy_calculator()
M_STTI.ROC_PR_curve_homemade()
param_M = M_STTI.ROC_PR_curve_sklearn()
df_M =  M_STTI.Classification_Report_index()

In [ ]:
CC_STTI = STTI.Statistic_reader(path_petastorm,Fiducial_metrics.Corr_lead_score,"Interlead",[-1,3],opposite = True)
CC_STTI.CrossValidation_index_opt_thresh()
CC_STTI.Optimal_threshold_calculator()
CC_STTI.ROC_index()
CC_STTI.PR_index()
CC_STTI.Accuracy_calculator()
CC_STTI.ROC_PR_curve_homemade()
param_CC = CC_STTI.ROC_PR_curve_sklearn()
df_CC =  CC_STTI.Classification_Report_index()

In [ ]:
K_STTI = STTI.Statistic_reader(path_petastorm,Fiducial_metrics.Kurto_score,"Kurtosis",[-10,30],opposite = True)
K_STTI.CrossValidation_index_opt_thresh()
K_STTI.Optimal_threshold_calculator()
K_STTI.ROC_index()
K_STTI.PR_index()
K_STTI.Accuracy_calculator()
#K_STTI.ROC_PR_curve_homemade()
param_K = K_STTI.ROC_PR_curve_sklearn()

In [ ]:
SQA_STTI = STTI.Statistic_reader(path_petastorm,Our_SQA_method.SQA_method_score,"SQA",[-1,2])
SQA_STTI.CrossValidation_index_opt_thresh()
SQA_STTI.Optimal_threshold_calculator()
SQA_STTI.ROC_index()
SQA_STTI.PR_index()
SQA_STTI.Accuracy_calculator()
SQA_STTI.ROC_PR_curve_homemade()
param_SQA = SQA_STTI.ROC_PR_curve_sklearn()
df_SQA =  SQA_STTI.Classification_Report_index()

In [ ]:
##Model with feature selection made on original data : 
MRegu_STTI = STTI.Statistic_reader(path_petastorm,Our_SQA_method.Model_regularization,"L2 Regularization",[-1,3])
MRegu_STTI.CrossValidation_index_opt_thresh()
MRegu_STTI.Optimal_threshold_calculator()
MRegu_STTI.ROC_index()
MRegu_STTI.PR_index()
MRegu_STTI.Accuracy_calculator()
MRegu_STTI.ROC_PR_curve_homemade()
param_MRegu = MRegu_STTI.ROC_PR_curve_sklearn()
df_SQANTSD =  MRegu_STTI.Classification_Report_index()

In [ ]:
MJMIMI_STTI = STTI.Statistic_reader(path_petastorm,Our_SQA_method.Model_MI,"JMI/MI features selection",[-1,3])
MJMIMI_STTI.CrossValidation_index_opt_thresh()
MJMIMI_STTI.Optimal_threshold_calculator()
MJMIMI_STTI.ROC_index()
MJMIMI_STTI.PR_index()
MJMIMI_STTI.Accuracy_calculator()
MJMIMI_STTI.ROC_PR_curve_homemade()
param_MJMIMI = MJMIMI_STTI.ROC_PR_curve_sklearn()
df_MJMIMI =  MJMIMI_STTI.Classification_Report_index()

In [ ]:
HR_STTI = STTI.Statistic_reader(path_petastorm,Fiducial_metrics.HR_index_calculator,"HR",[-1,5],evaluation = "minimum")
HR_STTI.CrossValidation_index_opt_thresh()
HR_STTI.Optimal_threshold_calculator()
HR_STTI.ROC_index()
HR_STTI.PR_index()
HR_STTI.Accuracy_calculator()
HR_STTI.ROC_PR_curve_homemade()
param_HR = HR_STTI.ROC_PR_curve_sklearn()
df_HR =  HR_STTI.Classification_Report_index()

In [ ]:
MoRE_STTI = STTI.Statistic_reader(path_petastorm,MoRE_2011.MoRE_score,"MoRE",[0,100])
MoRE_STTI.CrossValidation_index_opt_thresh()
MoRE_STTI.Optimal_threshold_calculator()
MoRE_STTI.ROC_index()
MoRE_STTI.PR_index()
MoRE_STTI.Accuracy_calculator()
param_MoRE = MoRE_STTI.ROC_PR_curve_sklearn()
MoRE_STTI.ROC_PR_curve_homemade()


In [ ]:
ECGASSESS_STTI = STTI.Statistic_reader(path_petastorm,ECG_Assess_2022.processing,"ECGASSESS",[-1,5])
ECGASSESS_STTI.CrossValidation_index_opt_thresh()
ECGASSESS_STTI.Optimal_threshold_calculator()
ECGASSESS_STTI.ROC_index()
ECGASSESS_STTI.PR_index()
ECGASSESS_STTI.Accuracy_calculator()
ECGASSESS_STTI.ROC_PR_curve_homemade()
param_ECGASSESS = ECGASSESS_STTI.ROC_PR_curve_sklearn()

In [ ]:
##Now to get the overall Training and Testing PR and ROC curve for all the indexes : 

get_name_dict = list(param_wPMF.keys())

name = ["wPMF","SNRECG","TSD","Flatline","HR","Intralead","Interlead","SQA"]
dic_gen = {name[0] : param_wPMF,name[1]:param_SNR_N,name[2]:param_TSD,name[3]:param_f,name[4]:param_HR,name[5]:param_M,name[6]:param_CC,name[7]:param_SQA}


fig,ax = plt.subplots(nrows = 2,ncols = 1,figsize= (15,15))

##Training Graphs : both PR and ROC :

x = np.linspace(0,1,200)
color = iter(plt.cm.rainbow(np.linspace(0, 1, len(name))))

for na,i in zip(name,range(len(name))) : 
    c = next(color)

    FPR_test_ind = dic_gen[na][get_name_dict[0]][0,:]
    TPR_test_ind = dic_gen[na][get_name_dict[0]][1,:]
    PREC_test_ind = dic_gen[na][get_name_dict[1]][1,:]
    REC_test_ind = dic_gen[na][get_name_dict[1]][0,:]

    auc_test_roc_ind = dic_gen[na][get_name_dict[2]]
    auc_test_roc_std = dic_gen[na][get_name_dict[3]]
    auc_test_pr_ind = dic_gen[na][get_name_dict[4]]
    auc_test_pr_std = dic_gen[na][get_name_dict[5]]
    
    ax[0].plot(FPR_test_ind,TPR_test_ind,color = c,label = "{} : AUC = {:.2f}(+- {:.2f})".format(na,auc_test_roc_ind,auc_test_roc_std))
    ax[1].plot(REC_test_ind,PREC_test_ind,color = c,label = "{} : AUC = {:.2f}(+- {:.2f})".format(na,auc_test_pr_ind,auc_test_pr_std))


ax[0].plot(x,x,"--k",label = "Reference line")
ax[0].set_xlabel("False Positive Rate")
ax[0].set_ylabel("True Positive Rate")
ax[0].set_title(f"Testing mean ROC Curve for all indexes created ")
ax[0].legend(loc = 4)
ax[0].grid()

ax[1].plot(x,np.zeros_like(x),"--k",label = "Reference line")
ax[1].set_xlabel("Recall")
ax[1].set_ylabel("Precision")
ax[1].set_title(f"Testing mean PR Curve for all indexes created ")
ax[1].legend(loc = 4)
ax[1].grid()
#bbox_to_anchor=(1.04, 1), loc="upper left"
plt.show()





In [ ]:
##Generalized previous function : 

def ROC_PR_printer_class(dico_class):
    fig,ax = plt.subplots(nrows = 1,ncols = 2,figsize= (15,10))
    ##Training Graphs : both PR and ROC : 
    x = np.linspace(0,1,200)
    color = iter(plt.cm.rainbow(np.linspace(0, 1, len(list(dico_class.keys())))))
    for na,i in zip(list(dico_class.keys()),range(len(list(dico_class.keys())))) : 
        c = next(color)
        FPR_test_ind = dico_class[na][get_name_dict[0]][0,:]
        TPR_test_ind = dico_class[na][get_name_dict[0]][1,:]
        PREC_test_ind = dico_class[na][get_name_dict[1]][1,:]
        REC_test_ind = dico_class[na][get_name_dict[1]][0,:]

        auc_test_roc_ind = dico_class[na][get_name_dict[2]]
        auc_test_roc_std = dico_class[na][get_name_dict[3]]
        auc_test_pr_ind = dico_class[na][get_name_dict[4]]
        auc_test_pr_std = dico_class[na][get_name_dict[5]]
    
        ax[0].plot(FPR_test_ind,TPR_test_ind,color = c,label = "{} : AUC = {:.2f}(+- {:.2f})".format(na,auc_test_roc_ind,auc_test_roc_std))
        ax[1].plot(REC_test_ind,PREC_test_ind,color = c,label = "{} : AUC = {:.2f}(+- {:.2f})".format(na,auc_test_pr_ind,auc_test_pr_std))



    ax[0].plot(x,x,"--k",label = "Reference line")
    ax[0].set_xlabel("False Positive Rate")
    ax[0].set_ylabel("True Positive Rate")
    ax[0].set_title(f"Testing mean ROC Curve for all indexes created ")
    ax[0].legend(loc = 4)
    ax[0].grid()

    ax[1].plot(x,np.zeros_like(x),"--k",label = "Reference line")
    ax[1].set_xlabel("Recall")
    ax[1].set_ylabel("Precision")
    ax[1].set_title(f"Testing mean PR Curve for all indexes created ")
    ax[1].legend(loc = 4)
    ax[1].grid()
    #bbox_to_anchor=(1.04, 1), loc="upper left"
    plt.show()


In [ ]:
get_name_dict = list(param_SQA.keys())

name = ["SQA","Model L2 Regularization","JMI/MI features selection"]
dic_gen = {name[0] : param_SQA,name[1]:param_MRegu,name[2]:param_MJMIMI}
ROC_PR_printer_class(dic_gen)
